<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/Lisanne_SVM/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib

In [0]:
# General packages
import numpy as np 
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import impute
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


# import classifier SVC
from sklearn.svm import SVC


In [17]:
# Load data
from adni.load_data import load_data
data = load_data()

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] # AD en CN moeten alleen nog omgezet worden naar 0 en 1
Y.replace(('AD', 'CN'), (1, 0), inplace=True) # AD en CN omgezet naar 0 en 1
print(X.shape)
print(Y.shape)
print(X)


The number of samples: 855
The number of columns: 268
(855, 267)
(855,)
                     hf_energy  ...  vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0
ID                              ...                                         
0_002_S_0413_bl_0  5403.229408  ...                             4.207831e-10
0_002_S_0559_bl_0   922.459760  ...                             3.137258e-10
0_002_S_0619_bl_0  1674.765276  ...                             4.219847e-10
0_002_S_0685_bl_0  3443.797200  ...                             5.954654e-11
0_002_S_0816_bl_0   914.469846  ...                             4.174750e-10
...                        ...  ...                                      ...
0_941_S_4292_bl_0  1503.564446  ...                             2.792072e-10
0_941_S_4365_bl_0  1087.136508  ...                             3.406526e-10
0_941_S_4376_bl_0  1963.323949  ...                             1.294541e-10
0_941_S_5124_bl_0  2392.983035  ...                             1.737102e-10
0_94

In [18]:
data

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,label,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_002_S_0413_bl_0,5403.229408,4.679203,2.392463,2.040475,1.375697,1.487644,-0.062859,429,0.364471,2.103334,-1.481394,0.455533,1,6395.491754,9940.032401,14622.897328,4.072445,5.204894,4.841280,3.774545,-0.108664,2.224809,0.498333,0.746542,0.785263,-0.007726,-0.069990,-0.029403,-0.000075,-0.068204,0.008103,-0.632451,-1.092715,-1.402496,1304,210,608,0.153782,0.627844,0.457846,...,0.020347,0.022628,0.000556,0.018271,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.446356e-17,3.969563,4.283295,1.930203e-09,4.762307e-10,3.147900e-10,1.195016e-10,8,4.503026e-10,1.810702e-09,2.003542,4.207831e-10
0_002_S_0559_bl_0,922.459760,4.772376,1.709731,1.437500,0.863074,0.940709,-0.247543,412,0.347258,1.685042,-1.222797,0.377850,1,8432.620063,5549.455518,12050.086848,4.076190,5.228020,4.830720,3.252139,-0.730229,0.348783,0.483188,0.412527,0.529344,-0.001275,-0.127039,-0.054140,0.000178,-0.145082,-0.044920,-0.578344,-0.633490,-0.828795,906,229,317,0.189966,0.455902,0.447878,...,0.010227,0.039959,0.001927,0.009183,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,2.058198e-17,4.401233,2.048344,1.189167e-09,3.678934e-10,3.325169e-10,6.933179e-13,11,4.3

In [19]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(598, 267) (598,)
(257, 267) (257,)


In [20]:
# Preprocessing: drop feature if too many missing values

X_train = X_train.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_train.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_train)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_train.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_train.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_train.shape, y_train.shape)

94
(598, 261) (598,)


In [21]:
# nu ook voor X_test
X_test = X_test.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_test.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_test)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_test.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_test.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_test.shape, y_test.shape)

46
(257, 261) (257,)


In [22]:
X_train

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_094_S_1027_bl_0,1603.106467,4.733283,1.892152,1.539706,1.017300,1.086302,-0.006832,274,0.350430,1.546538,-1.247318,0.346712,6498.377775,9267.864471,8140.219834,3.821292,4.943006,5.087486,4.766720,0.489231,0.688297,0.472199,0.538593,0.697258,-0.009305,-0.081316,-0.026169,-0.000014,-0.056175,0.026085,-0.606462,-1.112239,-1.249637,947,177,336,0.161015,0.531064,0.503152,1.078661,...,0.024000,0.025913,0.000564,0.022166,4.428750e-15,4.425793,3.129358,3.050972e-09,8.898321e-10,6.897234e-10,4.909738e-12,280,8.930833e-10,3.046062e-09,1.551065,7.650476e-10,4.428750e-15,4.425793,3.129358,3.050972e-09,8.898321e-10,6.897234e-10,4.909738e-12,280,8.930833e-10,3.046062e-09,1.551065,7.650476e-10,1.304920e-18,1.500000,-0.726467,9.820112e-10,3.455060e-10,1.443318e-10,6.306730e-11,2,3.418261e-10,9.189439e-10,1.085577,4.000928e-10
0_941_S_4376_bl_0,1963.323949,4.619950,2.012379,1.794024,1.167033,1.206801,0.168814,270,0.365045,1.625210,-1.009800,0.357589,10868.781917,6541.428230,8883.926398,3.210597,4.638562,4.062137,12.175568,1.555836,2.546862,0.478815,0.632926,0.642170,-0.012349,-0.001857,-0.015731,0.002733,0.016011,0.003032,-0.713178,-0.759681,-1.070406,1357,525,1180,0.064071,0.360431,0.210053,1.191993,...,0.041212,0.014895,0.000158,0.037882,6.041563e-15,4.201714,4.363515,3.991707e-09,9.517790e-10,6.619085e-10,1.928492e-16,508,1.018014e-09,3.991706e-09,1.879926,9.739068e-10,6.041563e-15,4.201714,4.363515,3.991707e-09,9.517790e-10,6.619085e-10,1.928492e-16,508,1.018014e-09,3.991706e-09,1.879926,9.739068e-10,3.907584e-18,4.409009,1.503238,4.483208e-10,1.257173e-10,8.605877

In [23]:
# Preprocessing:  impute missing values (fill missing)

imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_train_imp = imputer.fit_transform(X_train)          # impute  
X_train_imp = pd.DataFrame(data=X_train_imp, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

type(X_train_imp)

# en nu ook voor x_test
imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_test_imp = imputer.fit_transform(X_test)          # impute  
X_test_imp = pd.DataFrame(data=X_test_imp, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df

type(X_test_imp)

pandas.core.frame.DataFrame

In [24]:
# Preprocessing: scaling either standard or robust (removes median and scales data according to quantile range) 

scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_train_imp)
X_train_scaled = scaler.transform(X_train_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df
X_train_scaled

# nu ook voor X_TEST
scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_test_imp)
X_test_scaled = scaler.transform(X_test_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_test_scaled = pd.DataFrame(data=X_test_scaled, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df
X_test_scaled

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_041_S_5244_bl_0,0.193853,-0.634348,0.690581,1.205336,0.885126,0.733435,0.428507,0.672843,-0.415440,0.458189,-0.123324,-0.210857,0.526850,0.041188,0.381715,-0.881509,-0.658922,-1.457752,0.939295,0.643738,0.831592,0.155916,0.025955,-0.230510,-0.213802,0.435007,0.187491,-0.071465,0.417560,-0.125625,-0.023902,0.068862,0.011683,0.216724,1.283431,1.868434,-1.431846,-1.102915,-1.218536,0.081957,...,-0.230822,-6.984496e-01,-5.490876e-01,-2.768241e-01,-0.183445,0.216502,-0.407473,-0.150873,-0.164462,-0.188899,-0.665150,0.264953,0.179541,-0.126541,-0.342977,-0.057178,-0.183445,0.216502,-0.407473,-0.150873,-0.164462,-0.188899,-0.665150,0.264953,0.179541,-0.126541,-0.342977,-0.057178,-0.303771,1.024886,-0.307441,-0.538785,-0.527425,-0.422882,-0.658712,-0.279886,-0.271275,-0.513618,-0.431934,-0.569647
0_020_S_0883_bl_0,-0.498441,0.099657,0.043287,0.204040,0.232236,0.304119,-0.308481,-0.831763,-0.308451,0.527125,-0.745003,0.034205,-1.074088,-0.775185,-1.098909,0.488767,-0.051853,1.002057,-0.527658,0.088930,-1.144134,-0.705032,1.599812,-0.136875,0.564008,0.660341,0.196136,1.059067,0.461286,0.838846,1.310319,-0.086313,0.955198,-1.160680,-0.679045,-1.164404,0.343913,0.619852,0.825978,-1.217311,...,-0.230822,-3.360107e-01,-4.848315e-01,-3.439627e-01,-0.230931,0.448953,-0.307821,0.323872,0.429919,0.409108,0.747995,-0.997431,0.147570,0.298361,0.009645,0.330337,-0.230931,0.448953,-0.307821,0.323872,0.429919,0.409108,0.747995,-0.997431,0.147570,0.298361,0.009645,0.330337,-0.298903,0.139212,2.123271,0.921649,-0.127667,-0.217284,-0.117984,-0.433748,-0.563004,0.940194,2.391081

# We chose a RBF classifier, in the assignment there are 4 'forms' of RBF classifiers. 

# Option 1: use the standard RBF kernel from the scikit-learn SVM       #leek het niet te doen op de borst data set
clf = SVC(kernel='rbf', gamma=1)
clf.fit(X, Y)
y_pred = clf.predict(X)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

# Option 2: Manually transform the feature space with an RBF kernel, sample a part of this, than apply a linear SVM
rbf_feature = RBFSampler(gamma=1, random_state=1)
clf = SVC(kernel='linear')
XR = rbf_feature.fit_transform(X)
clf.fit(XR, Y)
y_pred = clf.predict(XR)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))
    
# Option 3: supply a manually constructed kernel function to the SVM     #leek het niett te doen op de borst data set
clf = SVC(kernel=rbf_kernel)
clf.fit(X, Y)
y_pred = clf.predict(X)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

# Option 4: precompute the RBF kernel                                    #leek het niet te doen op de borst data set
clf = SVC(kernel='precomputed')
rbf_kernel_computed = rbf_kernel(X)
clf.fit(rbf_kernel_computed, Y)
y_pred = clf.predict(rbf_kernel_computed)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

De SVM classifier

In [0]:
# Set parameters
parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-2],
                     'C': [1, 10]}]
                    
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

                    #in de exercises wordt voor gamma 'scale' of '1' gebruikt. 

scores = ['precision', 'recall']

In [40]:
#Grid search with cross-validation
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    svc = SVC()
    grid_search = GridSearchCV(
        svc, parameters, scoring='%s_macro' % score
    ) 
    # cv is nu default, dus een kfold van 5! 
    grid_search.fit(X_train_scaled, y_train)

    print("Best parameters set found on development set:")
    print()
    clf = grid_search.best_params_
    print(clf)
    print()
    print("Grid scores on development set:")
    print()
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

# test data
      #nog kijken hoe die dit automatisch kan doen.
clf = SVC(C=1, gamma=0.001, kernel='rbf')

model = clf.fit(X_train_scaled, y_train)
predictions = clf.predict(X_test_scaled)

print("score:")
print(model.score(X_test_scaled, y_test))
print(accuracy_score(y_test, predictions))

# y_true, y_pred = y_test, clf.predict(X_test_scaled)

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy:{accuracy}')
auc=metrics.roc_auc_score(y_true, y_pred)
print(f'AUC:{auc}')
accuracy=metrics.accuracy_score(y_true, y_pred)
print(f'accuracy:{auc}')
F1=metrics.f1_score(y_true,y_pred)
print(f'F1:{auc}')
precision=metrics.precision_score(y_true,y_pred)
print(f'precision:{auc}')
recall=metrics.recall_score(y_true, y_pred)
print(f'recall:{auc}')


# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.803 (+/-0.064) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.027) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.802 (+/-0.052) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.790 (+/-0.021) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.780 (+/-0.067) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.771 (+/-0.030) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.785 (+/-0.049) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.771 (+/-0.025) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

score:
0.8015564202334631
0.8015564202334631
Accuracy:0.9364548494983278
AUC:0.9251517273576098
accuracy:0.9251517273576098
F1:0.9251517273576098
precision:0.9251517273